<center><img src="https://www.mindinventory.com/blog/wp-content/uploads/2019/04/python-development-1200x500.png" width="1000"></center>

# Programa de Especialización en Python

## Tema 8. Análisis de Discriminante Lineal: Caso Practico

### Prof. Manuel Sigüeñas, M.Sc.(c)

In [31]:
import pandas as pd

In [32]:
import pandas as pd
import numpy as np

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score

%matplotlib inline

### Cargar los datos

Porcentajes diarios de rendimiento para el índice bursátil S&P 500 entre 2001 y 2005.

- **Year**: el año en que se registró la observación.
- **Lag1**: porcentaje de devolución del día anterior
- **Lag2**: porcentaje de devolución de 2 días anteriores
- **Lag3**: Porcentaje de devolución de 3 días anteriores
- **Lag4**: porcentaje de devolución de 4 días anteriores
- **Lag5**: porcentaje de devolución de 5 días anteriores
- **Volume**: volumen de acciones negociadas (número de acciones diarias negociadas en miles de millones)
- **Today**: porcentaje de retorno para hoy
- **Direction**: un factor con niveles hacia abajo y hacia arriba que indica si el mercado tuvo un rendimiento positivo o negativo en un día determinado

En Python, podemos ajustar un modelo LDA usando la función LinearDiscriminantAnalysis(), que es parte del módulo discriminant_analysis de la librería de ML sklearnbiblioteca. Ajustaremos el modelo utilizando solo las observaciones anteriores a 2005, y luego probaremos el modelo con los datos de 2005.

In [33]:
df = pd.read_csv('D:/Python/2. Nivel II/8/datos/Smarket.csv', usecols=range(1,10), index_col=0, parse_dates=True)
df.head()

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
Year,,,,,,,,
2001-01-01,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
2001-01-01,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2001-01-01,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
2001-01-01,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
2001-01-01,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


## LDA

In [34]:
X_train = df[:'2004'][['Lag1','Lag2']]
y_train = df[:'2004']['Direction']

X_test = df['2005':][['Lag1','Lag2']]
y_test = df['2005':]['Direction']

lda = LinearDiscriminantAnalysis()
model = lda.fit(X_train, y_train)

print(model.priors_)

[0.49198397 0.50801603]


La salida LDA indica probabilidades previas de $\hat{π}_1=0.492$ y $\hat{π}_2=0.508$; n otras palabras, el 49.2% de las observaciones de capacitación corresponden a días durante los cuales el mercado bajó.

In [35]:
print(model.means_)

[[ 0.04279022  0.03389409]
 [-0.03954635 -0.03132544]]


Lo anterior proporciona los medios grupales; estos son el promedio de cada predictor dentro de cada clase, y son utilizados por LDA como estimaciones de  $μ_k$. Esto sugiere que existe una tendencia a que los retornos de los 2 días anteriores sean negativos en los días en que el mercado aumenta, y una tendencia a que los retornos de los días anteriores sean positivos en los días en que el mercado declina.

In [36]:
print(model.coef_)

[[-0.05544078 -0.0443452 ]]


Los coeficientes de salida discriminantes lineales proporcionan la combinación lineal de Lag1y Lag2que se utilizan para formar la regla de decisión LDA.

$If −0.0554×Lag1−0.0443×Lag2$ es grande, entonces el clasificador LDA predecirá un aumento del mercado, y si es pequeño, entonces el clasificador LDA pronosticará una disminución del mercado. 



In [37]:
pred=model.predict(X_test)
print(np.unique(pred, return_counts=True))

(array(['Down', 'Up'], dtype='<U4'), array([ 70, 182], dtype=int64))


El modelo asignó 70 observaciones a la clase "Down" y 182 observaciones a la clase "Up". Veamos la matriz de confusión para ver cómo le está yendo a este modelo. Queremos comparar la clase predicha (que podemos encontrar en pred) con la clase verdadera (que se encuentra en $y_test$)

### Matriz de confusión

In [38]:
print(confusion_matrix(pred, y_test))

[[ 35  35]
 [ 76 106]]


In [39]:
confusion_matrix = pd.crosstab(pred, y_test)
confusion_matrix

Direction,Down,Up
row_0,,
Down,35,35
Up,76,106


In [17]:
print(classification_report(y_test, pred, digits=3))

              precision    recall  f1-score   support

        Down      0.500     0.315     0.387       111
          Up      0.582     0.752     0.656       141

    accuracy                          0.560       252
   macro avg      0.541     0.534     0.522       252
weighted avg      0.546     0.560     0.538       252



## QDA

In [12]:
qda = QuadraticDiscriminantAnalysis()
model2 = qda.fit(X_train, y_train)
print(model2.priors_)

[0.49198397 0.50801603]


In [13]:
print(model2.means_)

[[ 0.04279022  0.03389409]
 [-0.03954635 -0.03132544]]


In [14]:
pred2=model2.predict(X_test)
print(np.unique(pred2, return_counts=True))

(array(['Down', 'Up'], dtype=object), array([ 50, 202], dtype=int64))


In [15]:
print(confusion_matrix(pred2, y_test))
print(classification_report(y_test, pred2, digits=3))

[[ 30  20]
 [ 81 121]]
              precision    recall  f1-score   support

        Down      0.600     0.270     0.373       111
          Up      0.599     0.858     0.706       141

    accuracy                          0.599       252
   macro avg      0.600     0.564     0.539       252
weighted avg      0.599     0.599     0.559       252

